<a href="https://colab.research.google.com/github/wwoskie/GRLS_parser/blob/main/GRLS_BS_parser_HIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

In [ ]:
def url_maker(drug_name, page_number = 1):
  url = 'https://grls.rosminzdrav.ru/GRLS.aspx?RegNumber=&MnnR=' + quote(drug_name) + '&lf=&TradeNmR=&OwnerName=&MnfOrg=&MnfOrgCountry=&isfs=0&isND=-1&regtype=&pageSize=10&order=RegDate&orderType=desc&pageNum=' + str(page_number)
  return url

In [ ]:
def soup_checker(r):
  soup_check = BeautifulSoup(r.content, 'html5lib')
  table_check = soup_check.find('tr', attrs = {'class':'hdr_flat'})
  return ('Данные не найдены' in str(table_check))

In [ ]:
def comp_counter(comp):
  if comp == '-':
    out = -1
  else:
    out = len(comp.split('+'))
  return out

In [ ]:
def GRLS_parser(drug_list, table_name = 'GRLS_table.tsv', table_create = True, dct_ret = True):
  colnames = ['Текст запроса', '№ п/п', 'Торговое наименование', 'Международное непатентованное наименование или группировочное (химическое) наименование', 'Форма выпуска', 'Наименование держателя или владельца регистрационного удостоверения лекарственного препарата', 'Страна держателя или владельца регистрационного удостоверения лекарственного препарата', 'Регистрационный номер', 'Дата государственной регистрации', 'Дата окончания действ. рег. уд.', 'Дата переоформления РУ', 'Состояние', 'Дата решения']

  dct_out = {}

  for drug in drug_list:
    dct_out[drug] = {}
    url = url_maker(drug, page_number = 1)
    r = requests.get(url)
    soup_check = soup_checker(r)
    if not soup_check:
      i = 1

      while not soup_checker(r):
        url = url_maker(drug, page_number = i)
        r = requests.get(url)
        if soup_checker(r):
          continue
        soup = BeautifulSoup(r.content, 'html5lib')
        table = soup.find('table', attrs = {'class':'ts1 qa-result-table'})

        for line in table.findAll('td'):
          line = str(line)
          row = line.split('>')[1].split('<')[0].strip()
          if line.startswith('<td align="center" width="1">'):
            key = row
            dct_out[drug][key] = [drug, row]
          else:
            dct_out[drug][key].append(row)

        i += 1

    else:
      dct_out[drug]['-1'] = [drug, '-1', 'Не найдено ни одного препарата'] + ['-']*10

  df = pd.DataFrame(columns = colnames)
  for drug in dct_out.keys():
    for key in dct_out[drug].keys():
      df = df.append(pd.Series(dct_out[drug][key], index = colnames), ignore_index = True)

  df['Действующих веществ'] = df['Международное непатентованное наименование или группировочное (химическое) наименование'].map(comp_counter)

  if table_create:
    df.to_csv(table_name, sep = '\t', index = False)

  if dct_ret:
    return (df, dct_out)
  else:
    return df

Пример работы на лекарствах от СМА:
для бранаплама и релдесемтива ожидаемо ничего не найдено, т.к. они еще слишком экспериментальные


In [ ]:
df_out, dct_out = GRLS_parser(['Онасемноген абепарвовек', 'Нусинерсен', 'Рисдиплам', 'Бранаплам', 'Релдесемтив'], table_name = 'SMN_GRLS.tsv')

In [ ]:
df_out

In [ ]:
dct_out

In [ ]:
cat_list = {#'Нуклеозидные ингибиторы обратной транскриптазы (НИОТ/NRTI)':#взято с http://rushiv.ru/drugs-table/ данные 2017 года, возможно появились новые действующие вещества
            #['Зидовудин',
             #'Диданозин',
             #'Ставудин',
             #'Ламивудин',
             #'Абакавир',
             #'Тенофовир',
             #'Эмтрицитабин',
             #'Фосфазид',
             #'Тенофовира алафенамид'],
            'Нуклеозидные ингибиторы обратной транскриптазы (НИОТ/NRTI). Комбинированные препараты.':
            ['Зидовудин+Ламивудин',
             'Абакавир+Ламивудин+Зидовудин',
             'Абакавир+Ламивудин',
             'Эмтрицитабин+Тенофовир',
             'Ламивудин+Фосфазид']#,
            #'Ненуклеозидные ингибиторы обратной транскриптазы (ННИОТ/NNRTI)':
            #['Эфавиренз',
             #'Невирапин',
             #'Этравирин',
             #'Рилпивирин',
             #'Доравирин',
             #'Элсульфавирин'],
            #'ННИОТ/NNRTI + НИОТ/NRTI. Комбинированные препараты.':
            #['Тенофовир+Эмтрицитабин+Эфавиренз',
             #'Рилпивирин+Тенофовир+Эмтрицитабин',
             #'Доравирин+Ламивудин+Тенофовир'],
            #'Ингибиторы протеазы (ИП/PIs)':
            #['Саквинавир',
             #'Ритонавир',
             #'Атазанавир',
             #'Фосампренавир',
             #'Типранавир',
             #'Дарунавир'],
            #'Ингибиторы протеазы (ИП/PIs). Комбинированные препараты.':
            #['Лопинавир+Ритонавир'],
            #'Ингибиторы интегразы (ИИ/II)':
            #['Ралтегравир',
             #'Долутегравир'],
            #'ИИ/II + НИОТ/NRTI. Комбинированные препараты':
            #['Кобицистат+Тенофовира алафенамид+Элвитегравир+Эмтрицитабин'],
            #'Ингибиторы слияния (фузии) (ИC/FIs)':
            #['Энфувиртид'],
            #'Ингибиторы (блокаторы) рецепторов':
            #['Маравирок']
            }

In [ ]:
colnames = ['Группа', 'Текст запроса', '№ п/п', 'Торговое наименование', 'Международное непатентованное наименование или группировочное (химическое) наименование', 'Форма выпуска', 'Наименование держателя или владельца регистрационного удостоверения лекарственного препарата', 'Страна держателя или владельца регистрационного удостоверения лекарственного препарата', 'Регистрационный номер', 'Дата государственной регистрации', 'Дата окончания действ. рег. уд.', 'Дата переоформления РУ', 'Состояние', 'Дата решения']
df_HIV = pd.DataFrame(columns = colnames)
dct_HIV = {}

In [ ]:
for group in cat_list.keys():#скрипту нужно около 5 минут, чтобы пробежаться по всему вичовому словарю
  df_out, dct_out = GRLS_parser(cat_list[group], table_create = False)
  df_out['Группа'] = group
  dct_HIV[group] = dct_out
  df_HIV = df_HIV.append(df_out)

df_HIV.to_csv('HIV_GRLS2.tsv', sep = '\t', index = False)

In [ ]:
df_HIV

In [1]:
#dct_HIV